# Experiments

In [ ]:
dataset = 'citeseer'
normalize_features = True
train_samples_per_class = 30
seed = 24

lr = 0.001
epochs = 1000
K = 6
dropout = 0.2
weight_decay = 5e-4
hidden = 50
embed_dim = 50
num_cluster_iter = 1
clustertemp = 70

## Global arguments

In [ ]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
from models import GCNLink, GCNClusterNet, GCNDeep, GCNDeepSigmoid, GCN, GCNLinear
from utils import make_normalized_adj, negative_sample, load_nofeatures, accuracy, calculate_accuracy

import matplotlib.pyplot as plt

In [ ]:
if(normalize_features):
    from pygcn import load_data
else:
    from utils import load_data

In [ ]:
no_cuda = True

train_pct = 0.40
cuda = not no_cuda and torch.cuda.is_available()

np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    cuda.manual_seed(seed)

## Preparations

### Load data

In [ ]:
adj_test, features_test, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(dataset), '{}_test_{:.2f}'.format(dataset, train_pct))
adj_valid, features_valid, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(dataset), '{}_valid_{:.2f}'.format(dataset, train_pct))
adj_train, features_train, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(dataset), '{}_train_{:.2f}'.format(dataset, train_pct))

adj_test = adj_test.coalesce()
adj_valid = adj_valid.coalesce()
adj_train = adj_train.coalesce()
n = adj_train.shape[0]
bin_adj_test = (adj_test.to_dense() > 0).float()
bin_adj_train = (adj_train.to_dense() > 0).float()

bin_adj_valid = (adj_valid.to_dense() > 0).float()
bin_adj_all = (bin_adj_train + bin_adj_test + bin_adj_valid > 0).float()
adj_all = make_normalized_adj(bin_adj_all.nonzero(), n)
nfeat = features_test.shape[1]

adj_all, features_test, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(dataset), '{}'.format(dataset))
adj_all = adj_all.coalesce()
adj_test = adj_all

nfeat = features_test.shape[1]

### Select samples to train on

In [ ]:
train_label_indices = []

for i in range(K):
    for _ in range(train_samples_per_class):
        idx = np.random.randint(labels.shape[0])
        while(idx in train_label_indices or labels[idx] != i):
            idx = np.random.randint(labels.shape[0])
        train_label_indices.append(idx)
test_label_indices = list(set(range(labels.shape[0])) - set(train_label_indices))

In [ ]:
def plot_if_possible(r=None, print_normalized=False):
    if features_train.shape[1] == 2:
        if normalize_features and not print_normalized:
            x = []
            y = []
            with open("data/%s/%s.content" % (dataset, dataset)) as ss:
                for line in ss:
                    _, xx, yy, _ = line.split(' ')
                    x.append(float(xx))
                    y.append(float(yy))
        else:
            x = features_train[:,0]
            y = features_train[:,1]
        
        if print_normalized:
            text = " normalized"
        else:
            text = ""
        if r is None:
            plt.scatter(x, y, c=labels, s=5)
            plt.title("The%s dataset with true labels" % text)
        else:
            predictions = r.argmax(dim=1)
            plt.scatter(x, y, c=predictions, s=5)
            plt.title("The%s dataset with predicted labels" % text)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()

In [ ]:
plot_if_possible()

In [ ]:
if normalize_features:
    plot_if_possible(print_normalized=True)

In [ ]:
if cuda:
    features = features.cuda()
    adj_train = adj_train.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

## Training

### Decision-Focused

In [ ]:
model_cluster = GCNClusterNet(nfeat=nfeat,
            nhid=hidden,
            nout=embed_dim,
            dropout=dropout,
            K=K,
            cluster_temp = clustertemp)
if cuda:
    model_cluster.cuda()

optimizer = optim.Adam(model_cluster.parameters(),
                       lr=lr, weight_decay=weight_decay)

accuracies = []

for t in range(epochs):
    mu, r, embeds, dist = model_cluster(features_train, adj_train, 1)
    loss = nn.functional.cross_entropy(r[train_label_indices], labels[train_label_indices])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (t==500):
        num_cluster_iter = 5
    accuracies.append(accuracy(r, labels).item())

model_cluster.training = False
mu, r, embeds, dist = model_cluster(features_train, adj_train, num_cluster_iter)
print("ClusterNet accuracy: ", accuracy(r[test_label_indices], labels[test_label_indices]).item())
plot_if_possible(r)

In [ ]:
plt.plot(accuracies)
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

### E2E GCN

In [ ]:
print('GCN from paper')

model_gcn = GCNDeep(nfeat=nfeat,
            nhid=hidden,
            nout=K,
            dropout=dropout,
            nlayers=2)

optimizer_gcn = optim.Adam(model_gcn.parameters(), lr = lr,
                            weight_decay = weight_decay)

accuracies = []

for t in range(epochs):
    r = model_gcn(features_train, adj_train)
    loss = nn.functional.nll_loss(r[train_label_indices], labels[train_label_indices])
    optimizer_gcn.zero_grad()
    loss.backward()
    optimizer_gcn.step()
    accuracies.append(accuracy(r, labels).item())
    
print("e2e gcn accuracy: ", accuracy(r[test_label_indices], labels[test_label_indices]).item())
plot_if_possible(r)

In [ ]:
plt.plot(accuracies)
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
print('linear GCN')

model_gcn = GCNLinear(nfeat=nfeat,
            nhid=hidden,
            nout=embed_dim,
            dropout=dropout,
            nlayers=2,
            K=K)

optimizer_gcn = optim.Adam(model_gcn.parameters(), lr = lr,
                            weight_decay = weight_decay)

accuracies = []

for t in range(epochs):
    r = model_gcn(features_train, adj_train)
    loss = nn.functional.nll_loss(r[train_label_indices], labels[train_label_indices])
    optimizer_gcn.zero_grad()
    loss.backward()
    optimizer_gcn.step()
    accuracies.append(accuracy(r, labels).item())
    
print("e2e gcn accuracy: ", accuracy(r[test_label_indices], labels[test_label_indices]).item())
plot_if_possible(r)

In [ ]:
plt.plot(accuracies)
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()